In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
import tensorflow as tf
import keras
import random
import backtesting
from backtesting import Strategy, Backtest

seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users','Target' ]

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = 22

class CNN_Class_S1(Strategy):
    def init(self):
        self.trade_position = None
        self.days_since_position = 0

    def next(self):
        if self.data.Predictions > 0 and self.trade_position != 1:
            self.buy()
            self.trade_position = 1
            self.days_since_position = 0
        elif self.data.Predictions == 0 and self.trade_position != 0:
            self.sell()
            self.trade_position = 0
            self.days_since_position = 0

        # Close position if it's been S2 for a day
        if self.days_since_position == 1:
            if self.trade_position == 1:
                self.sell()
            else:
                self.buy()
            self.trade_position = None

        self.days_since_position += 1

class CNN_Class_S2(Strategy):
    def init(self):
        self.trade_position = None

    def next(self):
        # Long position logic
        if self.data.Predictions > 0 and self.trade_position != 1:
            if self.trade_position == 0:  
                self.buy()
            self.buy()  # S2 long position
            self.trade_position = 1
        # Short position logic
        elif self.data.Predictions == 0 and self.trade_position != 0:
            if self.trade_position == 1:  
                self.sell()
            self.sell() 
            self.trade_position = 0


# Full Data #

In [3]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/BTC_FINAL_df_class.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM3DCLASS.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))

# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats = full_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)


full_stats.to_latex('lstm_full_stats.tex', float_format="%.2f")

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


# Bear Market #

## Bear Market 1 ##

In [4]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_1_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bear_market_1.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))

# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats_br1 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats_br1

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


,S1,S2
Start,2018-11-12 00:00:00,2018-11-12 00:00:00
End,2018-12-15 00:00:00,2018-12-15 00:00:00
Duration,33 days 00:00:00,33 days 00:00:00
Exposure Time [%],94.117647,94.117647
Equity Final [$],136874.972196,169978.729604
Equity Peak [$],145671.674608,169978.729604
Return [%],36.874972,69.97873
Buy & Hold Return [%],-49.197543,-49.197543
Return (Ann.) [%],2807.225419,29640.8632
Volatility (Ann.) [%],3579.607051,25564.933829


## Bear 2 ##

In [5]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_2_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bear_market_2.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))

# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()

# merge the stats
# Merge the stats
full_stats_br2 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats_br2

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


,S1,S2
Start,2022-10-17 00:00:00,2022-10-17 00:00:00
End,2022-11-21 00:00:00,2022-11-21 00:00:00
Duration,35 days 00:00:00,35 days 00:00:00
Exposure Time [%],94.444444,94.444444
Equity Final [$],80731.932383,115067.911805
Equity Peak [$],102428.615784,117123.170595
Return [%],-19.268068,15.067912
Buy & Hold Return [%],-19.249758,-19.249758
Return (Ann.) [%],-88.583234,314.961862
Volatility (Ann.) [%],8.662039,315.798022


In [6]:
# Merge the stats
lstm_bear_stats = pd.concat([full_stats_br1, full_stats_br2], axis=1, keys=['Bear_Market_1', 'Bear_Market_2'])


# Erase unecessary rows
lstm_bear_stats = lstm_bear_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)

# Aggregate the stats
# Calculate the averages for each strategy
lstm_bear_stats['S1_Avg'] = (lstm_bear_stats[('Bear_Market_1', 'S1')] + lstm_bear_stats[('Bear_Market_2', 'S1')]) / 2
lstm_bear_stats['S2_Avg'] = (lstm_bear_stats[('Bear_Market_1', 'S2')] + lstm_bear_stats[('Bear_Market_2', 'S2')]) / 2

lstm_bear_stats.to_latex('lstm_bear_stats.tex', float_format="%.2f", multicolumn=True)

lstm_bear_stats

Bear_Market_1                       Bear_Market_2  \
                                     S1                S2                S1   
Duration               33 days 00:00:00  33 days 00:00:00  35 days 00:00:00   
Return [%]                    36.874972          69.97873        -19.268068   
Buy & Hold Return [%]        -49.197543        -49.197543        -19.249758   
Return (Ann.) [%]           2807.225419        29640.8632        -88.583234   
Volatility (Ann.) [%]       3579.607051      25564.933829          8.662039   
Sharpe Ratio                   0.784227          1.159434               0.0   
# Trades                             32                 5                34   
Win Rate [%]                      56.25              60.0         44.117647   

                                                   S1_Avg            S2_Avg  
                                     S2                                      
Duration               35 days 00:00:00  34 days 00:00:00  34 days 00:00:00  
Return [%]                    15.067912          8.803452         42.523321  
Buy & Hold Return [%]        -19.249758        -34.223651        -34.223651  
Return (Ann.) [%]            314.961862       1359.321093      14977.912531  
Volatility (Ann.) [%]        315.798022       1794.134545      12940.365926  
Sharpe Ratio                   0.997352          0.392114          1.078393  
# Trades                              1              33.0               3.0  
Win Rate [%]                      100.0         50.183824              80.0

# Bull Market #

In [7]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_1_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bull_market_1.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))

# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()


# merge the stats
# Merge the stats
full_stats_bl1 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats_bl1

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


,S1,S2
Start,2017-09-04 00:00:00,2017-09-04 00:00:00
End,2017-12-17 00:00:00,2017-12-17 00:00:00
Duration,104 days 00:00:00,104 days 00:00:00
Exposure Time [%],98.113208,79.245283
Equity Final [$],108395.480195,0.0
Equity Peak [$],137549.25149,126730.330865
Return [%],8.39548,-100.0
Buy & Hold Return [%],351.827192,351.827192
Return (Ann.) [%],32.344381,0.0
Volatility (Ann.) [%],201.618489,32699.866962


## Bull Market 2 ##

In [8]:
seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_2_classification.csv')

# Load model
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bull_market_2.keras')

# keep only the columns we need
col = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target']
backtest_data = data[col]
training_size = int(len(data) * 0.9)
backtest_data = backtest_data[training_size:]

# Split data only keep test data

# MAke the date the index
data.set_index('Date', inplace=True)

# Normalize the data
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
test_data = data[training_size:]


# Create the dataset
X_test, y_test = create_dataset(test_data, window_size, close_index)

# Reshape the data
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))

# Make predictions
predictions = full_data_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

# create a df with 3 empty values (window size) and the predictions
test_pred = np.concatenate([np.zeros((window_size, 1)), test_pred])

# Add the predictions to the test data
backtest_data['Predictions'] = test_pred.astype(int)

# Erase the first 3 values
backtest_data = backtest_data[window_size:]

# set date as index
backtest_data.set_index('Date', inplace=True)
backtest_data.index = pd.to_datetime(backtest_data.index)

# S1 Max Strategy
bt1 = Backtest(backtest_data, CNN_Class_S1, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats1 = bt1.run()


# Morethan S1 strategy
bt2 = Backtest(backtest_data, CNN_Class_S2, cash=100000, commission=.0015, exclusive_orders=True, trade_on_close=True)
stats2 = bt2.run()


# merge the stats
# Merge the stats
full_stats_bl2 = pd.concat([stats1, stats2], axis=1, keys=['S1', 'S2'])

full_stats_bl2

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


,S1,S2
Start,2021-07-30 00:00:00,2021-07-30 00:00:00
End,2021-11-10 00:00:00,2021-11-10 00:00:00
Duration,103 days 00:00:00,103 days 00:00:00
Exposure Time [%],80.952381,78.095238
Equity Final [$],56291.571844,54603.527664
Equity Peak [$],107974.749527,106821.551102
Return [%],-43.708428,-45.396472
Buy & Hold Return [%],53.887508,53.887508
Return (Ann.) [%],-86.690791,-88.039599
Volatility (Ann.) [%],6.042747,9.58788


In [9]:
# Merge the stats
lstm_bull_stats = pd.concat([full_stats_bl1, full_stats_bl2], axis=1, keys=['Bull_Market_1', 'Bull_Market_2'])


# Erase unecessary rows
lstm_bull_stats = lstm_bull_stats.drop(['Start', 'End', 'Exposure Time [%]', 'Equity Final [$]', 'Equity Peak [$]',
                            'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
                            'Avg. Drawdown Duration', 'Max. Trade Duration', 'Avg. Trade Duration', 'SQN', '_strategy',
                            '_equity_curve', '_trades', 'Expectancy [%]','Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
                            'Profit Factor'],
                             axis=0)

# Aggregate the stats
# Calculate the averages for each strategy
lstm_bull_stats['S1_Avg'] = (lstm_bull_stats[('Bull_Market_1', 'S1')] + lstm_bull_stats[('Bull_Market_2', 'S1')]) / 2
lstm_bull_stats['S2_Avg'] = (lstm_bull_stats[('Bull_Market_1', 'S2')] + lstm_bull_stats[('Bull_Market_2', 'S2')]) / 2

lstm_bull_stats.to_latex('lstm_bull_stats.tex', float_format="%.2f", multicolumn=True)

lstm_bull_stats

Bull_Market_1                     \
                                      S1                 S2   
Duration               104 days 00:00:00  104 days 00:00:00   
Return [%]                       8.39548             -100.0   
Buy & Hold Return [%]         351.827192         351.827192   
Return (Ann.) [%]              32.344381                0.0   
Volatility (Ann.) [%]         201.618489       32699.866962   
Sharpe Ratio                    0.160424                0.0   
# Trades                             104                  1   
Win Rate [%]                   44.230769                0.0   

                           Bull_Market_2                     \
                                      S1                 S2   
Duration               103 days 00:00:00  103 days 00:00:00   
Return [%]                    -43.708428         -45.396472   
Buy & Hold Return [%]          53.887508          53.887508   
Return (Ann.) [%]             -86.690791         -88.039599   
Volatility (Ann.) [%]           6.042747            9.58788   
Sharpe Ratio                         0.0                0.0   
# Trades                              83                  1   
Win Rate [%]                   39.759036                0.0   

                                  S1_Avg             S2_Avg  
                                                             
Duration               103 days 12:00:00  103 days 12:00:00  
Return [%]                    -17.656474         -72.698236  
Buy & Hold Return [%]          202.85735          202.85735  
Return (Ann.) [%]             -27.173205           -44.0198  
Volatility (Ann.) [%]         103.830618       16354.727421  
Sharpe Ratio                    0.080212                0.0  
# Trades                            93.5                1.0  
Win Rate [%]                   41.994903                0.0

In [17]:
# Get hyperparameters of each model
# Load the models
full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM3DCLASS.keras')
bear_1 = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bear_market_1.keras')
bear_2 = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bear_market_2.keras')
bull_1 = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bull_market_1.keras')
bull_2 = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/LSTM/LSTM_CLASS/best_LSTM-3D-CLASS-bull_market_2.keras')

# Get the hyperparameters
lr = full_data_model.optimizer.learning_rate.numpy()

print(lr)

1e-04


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [27]:
import json

full_data_model = keras.models.load_model('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/CNN/CNN_REG/best_CNN3D.keras')
lr = full_data_model.optimizer.learning_rate.numpy()
con = full_data_model.get_config()
print(lr)
model_config_pretty = json.dumps(con, indent=4)
print(model_config_pretty)

0.001
{
    "name": "sequential_31",
    "trainable": true,
    "dtype": "float32",
    "layers": [
        {
            "module": "keras.layers",
            "class_name": "InputLayer",
            "config": {
                "batch_shape": [
                    null,
                    3,
                    22,
                    1
                ],
                "dtype": "float32",
                "sparse": false,
                "name": "input_layer_31"
            },
            "registered_name": null
        },
        {
            "module": "keras.layers",
            "class_name": "Conv2D",
            "config": {
                "name": "conv2d_93",
                "trainable": true,
                "dtype": "float32",
                "filters": 50,
                "kernel_size": [
                    3,
                    3
                ],
                "strides": [
                    1,
                    1
                ],
                "padding": "same